In [1]:
from moviepy.editor import VideoFileClip
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torchaudio
import torchaudio.transforms as T
import torch
import numpy as np
from evaluate import load
wer = load("wer")

# Trimming video

In [2]:
def trim_video(video_file_path, start_time, end_time, trimmed_video_path):
    video = VideoFileClip(video_file_path).subclip(start_time, end_time)
    video.write_videofile(trimmed_video_path, codec="libx264", audio_codec="aac")

In [3]:
# Path to your MP4 video file
video_file_path = '../data/video/VOD_08-12-2022_M__de_i_salen.mp4'
# Path for the trimmed video
trimmed_video_path = '../data/video/trimmed/snip.mp4'
# Path to save the extracted audio
audio_file_path = '../data/audio/snip.wav'

# Time window for trimming (in seconds)
start_time = 60 + 47 
end_time = 60 + 47 + 29

# Trim video
trim_video(video_file_path, start_time, end_time, trimmed_video_path)

Moviepy - Building video ../data/video/trimmed/snip.mp4.
MoviePy - Writing audio in snipTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video ../data/video/trimmed/snip.mp4



Moviepy - Done !
Moviepy - video ready ../data/video/trimmed/snip.mp4


# Extract audio

In [4]:
def extract_audio_from_video(video_file_path, audio_file_path):
    video = VideoFileClip(video_file_path)
    video.audio.write_audiofile(audio_file_path, codec='pcm_s16le')

# Extract audio from trimmed video
extract_audio_from_video(trimmed_video_path, audio_file_path)

MoviePy - Writing audio in ../data/audio/snip.wav


MoviePy - Done.


# Manipulate audio

In [5]:
# load in audio
waveform, sample_rate = torchaudio.load(audio_file_path)

# Convert stereo audio to mono by averaging the two channels
waveform_mono = torch.mean(waveform, dim=0, keepdim=True)

# Fix the sample rate
resampler = T.Resample(orig_freq=sample_rate, new_freq=16000)
waveform_resampled = resampler(waveform_mono)

# save the audio
torchaudio.save('../data/audio/resampled/snip.wav', waveform_resampled, 16000)

# delete variables
del waveform, waveform_mono, waveform_resampled, sample_rate, resampler

# Transcribe

In [27]:
# load in audio
waveform, sample_rate = torchaudio.load('../data/audio/resampled/snip.wav')
waveform_np = waveform.squeeze().numpy()

In [28]:
# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
model.config.forced_decoder_ids = None

# load data
input_features = processor(waveform_np, sampling_rate=sample_rate, return_tensors="pt").input_features 

# generate token ids
predicted_ids = model.generate(input_features)

# decode token ids to text
whisper_tiny = processor.batch_decode(predicted_ids, skip_special_tokens=True)

In [29]:
whisper_tiny

[' Tak for det, for åmen. Jeg er det rigtig, det er beslutningst forslade b4, om opfølgelig en provisere i ureformgruppens andefalinger endringer, færdig udvalderer i konnen, så der er det skratskundlade. Og hvis man starter først med EU-reformgruppen, så er det i UFTV i juni 2000er tøb beslutet i Europa-delet og folgetændende og næsset, en ekspertgruppet, skulle klike på, at der er en vi arbejde med EU-sererne, her, hvor det er faktisk i disse måneder,']

# Hviske

In [30]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

processor = AutoProcessor.from_pretrained("mhenrichsen/hviske")
model = AutoModelForSpeechSeq2Seq.from_pretrained("mhenrichsen/hviske")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.32s/it]


In [31]:
# load data
input_features = processor(waveform_np, sampling_rate=sample_rate, return_tensors="pt").input_features 

# generate token ids
predicted_ids = model.generate(input_features)
# decode token ids to text
hviske = processor.batch_decode(predicted_ids, skip_special_tokens=True)

In [32]:
hviske

['Tak for det formand.Ja, det er rigtigt, det er beslutningsforslag B4 om opfølgningen på visse af euroreformgruppens anbefalinger og ændringer af ferieudvalget og gønlænsudvalgets retsgrundlag.Og hvis man starter først med euroreformgruppen, så er det jo, fordi vi i juni to tusind og tyve besluttede europaudvalget og folketinget, at nedsætte en ekspertgruppe, der skulle kigge på, hvordan vi arbejdede med euro-sagerne, her hvor det faktisk i disse måneder er halvtreds']

# Check quality

In [35]:
original_transcript = ['Tak for det, formand. Og ja, det er rigtigt, at vi nu behandler beslutningsforslag B 4 om opfølgning på visse af EU-reformgruppens anbefalinger og ændring af Færøudvalgets og Grønlandsudvalgets retsgrundlag. Hvis man starter med EU-reformgruppen, kan man nævne, at vi i Europaudvalget og i Folketinget i juni 2020 besluttede at nedsætte en ekspertgruppe, der skulle kigge på, hvordan vi arbejdede med EU-sagerne.']

In [36]:
actual_spoken = ['Tak for det, formand. Og ja, det er rigtigt, det er beslutningsforslag B 4 om opfølgning på visse af EU-reformgruppens anbefalinger og ændring af Færøudvalgets og Grønlandsudvalgets retsgrundlag. Og hvis man starter først med EU-reformgruppen, så er det jo  fordi vi i juni to tusind og tyve besluttede Europaudvalget og Folketinget og nedsætte en ekspertgruppe, der skulle kigge på, hvordan vi arbejdede med EU-sagerne. Her hvor det er faktisk i disse måneder at halv']

In [37]:
print(f'Whisper tiny WER: {round(100 * wer.compute(references=original_transcript, predictions=whisper_tiny),2)}')
print(f'Hviske WER:       {round(100 * wer.compute(references=original_transcript, predictions=hviske),2)}')

Whisper tiny WER: 98.39
Hviske WER:       69.35


In [38]:
print(f'Whisper tiny WER: {round(100 * wer.compute(references=actual_spoken, predictions=whisper_tiny),2)}')
print(f'Hviske WER:       {round(100 * wer.compute(references=actual_spoken, predictions=hviske),2)}')

Whisper tiny WER: 66.22
Hviske WER:       31.08


In [42]:
print(f'Original transcript: {original_transcript}')
print(f'Actual spoken:       {actual_spoken}')
print(f'Hviske:              {hviske}')
print(f'Whisper tiny:        {whisper_tiny}')

Original transcript: ['Tak for det, formand. Og ja, det er rigtigt, at vi nu behandler beslutningsforslag B 4 om opfølgning på visse af EU-reformgruppens anbefalinger og ændring af Færøudvalgets og Grønlandsudvalgets retsgrundlag. Hvis man starter med EU-reformgruppen, kan man nævne, at vi i Europaudvalget og i Folketinget i juni 2020 besluttede at nedsætte en ekspertgruppe, der skulle kigge på, hvordan vi arbejdede med EU-sagerne.']
Actual spoken:       ['Tak for det, formand. Og ja, det er rigtigt, det er beslutningsforslag B 4 om opfølgning på visse af EU-reformgruppens anbefalinger og ændring af Færøudvalgets og Grønlandsudvalgets retsgrundlag. Og hvis man starter først med EU-reformgruppen, så er det jo  fordi vi i juni to tusind og tyve besluttede Europaudvalget og Folketinget og nedsætte en ekspertgruppe, der skulle kigge på, hvordan vi arbejdede med EU-sagerne. Her hvor det er faktisk i disse måneder at halv']
Hviske:              ['Tak for det formand.Ja, det er rigtigt, det e